In [ ]:
from pylab import *
import os
import sys
from mixture import GaussianMixture
import pickle
from PIL import Image

In [ ]:
dataset_dir = 'rgbd_dataset_freiburg3_long_office_household'
depth_dir = 'depth'
gmm_dir = 'gmm_qqvga_mine_patch'

# og 
fx = 525.0  # focal length x
fy = 525.0  # focal length y
cx = 319.5  # optical center x
cy = 239.5  # optical center y

# fri3 
fx = 535.4  # focal length x
fy = 539.2  # focal length y
cx = 320.1  # optical center x
cy = 247.6  # optical center y
factor = 5000 # for the 16-bit PNG files
# OR: factor = 1 # for the 32-bit float images in the ROS bag files
baseline = 0.075 

full_depth_dir = os.path.join(dataset_dir,depth_dir)
full_gmm_dir = os.path.join(dataset_dir,gmm_dir)
if not os.path.exists(full_gmm_dir):
    os.mkdir(full_gmm_dir)

In [ ]:
max_disp = 325 #fx*baseline/0.125
min_disp = 2#f


In [ ]:
y_grid = np.repeat(np.arange(480)[:,None],640,1)
x_grid = np.repeat(np.arange(640)[None,:],480,0)


def img_to_pts(img):
    Z = img/factor
    X = (x_grid-cx) * Z /fx
    Y = (y_grid-cy) * Z /fy
    subsample = 4

    #uniform dist
    covar_X = (subsample* 1 * Z /fx)**2 * (1.0/12.0)
    covar_Y = (subsample* 1 * Z /fy)**2 * (1.0/12.0) 
    covar_Z = (0 * Z /fx)**2 * (1.0/12.0)
    covar = np.zeros((480,640,3,3))
    covar[:,:,0,0] = covar_X
    covar[:,:,1,1] = covar_Y
    covar[:,:,2,2] = covar_Z

    covar = covar[::subsample,::subsample,:,:]

    X = X[::subsample,::subsample]
    Y = Y[::subsample,::subsample]
    Z = Z[::subsample,::subsample]
    print(X.shape)

    xl = X[Z > 0]
    yl = Y[Z > 0]
    zl = Z[Z > 0]
    c1 = (covar.reshape((120,160,-1))[Z > 0]).reshape( (-1,3,3) )
    n = zl.shape[0]
    print(n,c1.shape)
    
    idx = np.random.randint(n, size=n//16)
    return np.vstack([xl,yl,zl]).T, c1

def load_data(file):
    image = Image.open(file)   
    pixel = np.array(image)
    return img_to_pts(pixel)

for fl in sorted(os.listdir(full_depth_dir)):
    name,ext = os.path.splitext(fl)
    new_name = name + '.pkl'
    
    clf = GaussianMixture(100)
    data = load_data(os.path.join(full_depth_dir,fl))
    print([_.shape for _ in data])
    t1 = time.time()
    
    n = data[0].shape[0]
    clf.set_areas(np.ones(n))
    clf.set_covars(data[1])
    clf.fit(data[0])
    clf.set_areas(None)
    clf.set_covars(None)
    
    print(time.time()-t1)
    with open(os.path.join(full_gmm_dir,new_name),'wb') as fp:
        pickle.dump(clf,fp)
    

In [ ]:
frtptB.shape